In [1]:
import json
import random

random.seed(42)

input_file_lima = "/data/miaopeng/workplace/LLaMA-Factory/data/lima/lima_train_full.jsonl"

# 加载输入数据
all_datas_lima = []
with open(input_file_lima, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        all_datas_lima.append(data)

In [2]:
training_queations = []

traing_file = "/data/miaopeng/workplace/LLaMA-Factory/6000Q_Assignment4/infer_results/lima_results.jsonl"
with open(traing_file, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        training_queations.append(data["question"])

# Random select 10 questions that are not in the training set
selected_questions = []
while len(selected_questions) < 10:
    random_index = random.randint(0, len(all_datas_lima) - 1)
    question = all_datas_lima[random_index]["question"]
    if question not in training_queations and question not in selected_questions:
        selected_questions.append(all_datas_lima[random_index])
        print(f"Selected question: {question}")

Selected question: How to set button click effect in Android?
Selected question: If user input is inserted without modification into an SQL query, then the application becomes vulnerable to SQL injection, like in the following example:
```$unsafe_variable = $_POST['user_input'];
mysql_query("INSERT INTO `table` (`column`) VALUES ('$unsafe_variable')");
```
That's because the user can input something like ```value'); DROP TABLE table;--```, and the query becomes:
```INSERT INTO `table` (`column`) VALUES('value'); DROP TABLE table;--')
```
What can be done to prevent this from happening?
Selected question: The "What if the Nazis won??" trope has been done to death. This time, imagine you live in a world where they won and write a story based on the prompt, "What if the allies won??"
Selected question: In India, we eat rice using our fingers. Generally in the West, a fork or spoon is used. I have tried eating rice with spoon but I don't feel satisfied with it.
We eat dry rice but we mix c

In [3]:
"""
Generate completions with base Mistral model
"""

import json
import os
import torch
import random
from tqdm import tqdm
from transformers import AutoModelForCausalLM
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

random.seed(42)
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

# 模型路径和文件路径
out_dir = "/data/miaopeng/workplace/LLaMA-Factory/6000Q_Assignment4"

# 初始化Mistral tokenizer
tokenizer = MistralTokenizer.v1()

# 初始化Mistral模型
model = AutoModelForCausalLM.from_pretrained("/data/miaopeng/workplace/LLaMA-Factory/hugging_cache/Mistral-7B-Instruct-v0.2")
model.to("cuda")

# 输出路径
output_dir = os.path.join(out_dir, "infer_results")
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "test_Mistral_base.jsonl")

# 遍历每一个数据条目
for idx, data in enumerate(tqdm(selected_questions, desc="Processing data")):

    # 构建Mistral prompt
    completion_request = ChatCompletionRequest(messages=[UserMessage(content=data['question'])])
    tokens = tokenizer.encode_chat_completion(completion_request).tokens

    # 使用Mistral模型生成结果
    tokens_tensor = torch.tensor([tokens]).to("cuda")
    generated_ids = model.generate(tokens_tensor, max_new_tokens=1000, do_sample=True)
    
    # 解码生成的结果
    result = tokenizer.decode(generated_ids[0].tolist())

    result = result.split("[/INST]")[-1].strip()
    data["base_prediction"] = result

    # 立即保存该条数据
    with open(output_file, 'a', encoding='utf-8') as f:
        f.write(json.dumps(data, ensure_ascii=False) + '\n')

print("Inference of Mistral completed and results saved.")

2025-05-02 18:55:38.010991: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-02 18:55:38.022824: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746183338.037697  687834 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746183338.042138  687834 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746183338.053186  687834 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Processing data:   0%|          | 0/10 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Processing data:  10%|█         | 1/10 [00:13<01:57, 13.07s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing data:  20%|██        | 2/10 [00:28<01:56, 14.55s/it]The attention mask and the pad token id were not set. As a consequence, you may observe un

Inference of Mistral completed and results saved.


In [4]:
"""
Generate completions with DPO fine-tuned model
"""

import json
import os
import torch
import random
from tqdm import tqdm
from transformers import AutoModelForCausalLM
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

random.seed(42)
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

# 模型路径和文件路径
out_dir = "/data/miaopeng/workplace/LLaMA-Factory/6000Q_Assignment4"

# 初始化Mistral tokenizer
tokenizer = MistralTokenizer.v1()

# 初始化Mistral模型
model = AutoModelForCausalLM.from_pretrained("/data/miaopeng/workplace/LLaMA-Factory/saves/mistral-7b-v0.2/lora/lima_dpo/checkpoint-33")
model.to("cuda")

# 输出路径
output_dir = os.path.join(out_dir, "infer_results")
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "test_Mistral_dpo.jsonl")

# 遍历每一个数据条目
for idx, data in enumerate(tqdm(selected_questions, desc="Processing data")):

    # 构建Mistral prompt
    completion_request = ChatCompletionRequest(messages=[UserMessage(content=data['question'])])
    tokens = tokenizer.encode_chat_completion(completion_request).tokens

    # 使用Mistral模型生成结果
    tokens_tensor = torch.tensor([tokens]).to("cuda")
    generated_ids = model.generate(tokens_tensor, max_new_tokens=1000, do_sample=True)
    
    # 解码生成的结果
    result = tokenizer.decode(generated_ids[0].tolist())

    result = result.split("[/INST]")[-1].strip()
    data["dpo_prediction"] = result

    # 立即保存该条数据
    with open(output_file, 'a', encoding='utf-8') as f:
        f.write(json.dumps(data, ensure_ascii=False) + '\n')

print("Inference of Mistral-DPO completed and results saved.")

2025-05-02 18:59:38.268413: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-02 18:59:38.281263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746183578.296529  688241 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746183578.301239  688241 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746183578.312868  688241 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Processing data:   0%|          | 0/10 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Processing data:  10%|█         | 1/10 [00:37<05:40, 37.80s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing data:  20%|██        | 2/10 [00:46<02:45, 20.75s/it]The attention mask and the pad token id were not set. As a consequence, you may observe un

Inference of Mistral-DPO completed and results saved.


In [6]:
import pandas as pd
import json
import os
import random
import numpy as np

random.seed(42)

input_file_base = "/data/miaopeng/workplace/LLaMA-Factory/6000Q_Assignment4/infer_results/test_Mistral_base.jsonl"
input_file_dpo = "/data/miaopeng/workplace/LLaMA-Factory/6000Q_Assignment4/infer_results/test_Mistral_dpo.jsonl"

# Load input data
all_datas_base = []
with open(input_file_base, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        all_datas_base.append(data)
        
all_datas_dpo = []
with open(input_file_dpo, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        all_datas_dpo.append(data)

# Create a list of dictionaries for the DataFrame
data_list = []
for idx, data in enumerate(all_datas_base):
    data_list.append({
        "question": data["question"],
        "base_prediction": data["base_prediction"],
        "dpo_prediction": all_datas_dpo[idx]["dpo_prediction"]
    })

# Create DataFrame from the list
df = pd.DataFrame(data_list)

# Save DataFrame to CSV file
output_csv_file = "/data/miaopeng/workplace/LLaMA-Factory/6000Q_Assignment4/infer_results/test_Mistral_DataFrame_Base_DPO.csv"
df.to_csv(output_csv_file, index=False)

# Print the first 10 rows of the DataFrame
print(df.head(10))

                                            question  \
0         How to set button click effect in Android?   
1  If user input is inserted without modification...   
2  The "What if the Nazis won??" trope has been d...   
3  In India, we eat rice using our fingers. Gener...   
4  How to deal with diagnosis of a rare chromosom...   
5  I am writing a story where a species undergoes...   
6  Betty has a tray of cookies and a tray of brow...   
7  Is there a Git for data? The key improvement I...   
8      How to treat ocd and anxiety as a workaholic?   
9   When is the best time to rob a convenience store   

                                     base_prediction  \
0  In Android, you can create a button click effe...   
1  To prevent SQL injection attacks, you should n...   
2  Title: "Parallel Lives: A World Saved"\n\nIn t...   
3  Eating rice with your hands in front of Wester...   
4  Receiving a diagnosis of a rare chromosome dis...   
5  In the context of your story, the concept of

### Compete display result can be reached at "data/infer_results/test_Mistral_DataFrame_Base_DPO.csv"